In [2]:
import pandas as pd
import numpy as np
import os

csv_path = os.path.join(os.getcwd(), '../raw_data', 'credit.csv')
      
data = pd.read_csv(csv_path)

data

,birth_date,credit_amount,salary_amount,region,gender,marital_status,credit_history,card_incomes_amount,target
0,08/12/1988,316315800,96800000,tashkent_city,M,married,no_credit_now,114000000,1
1,13/07/1994,237064800,180000000,tashkent_city,M,married,no_credit_now,194483333,1
2,18/12/1988,248900000,350000000,tashkent_city,M,not_married,no_credit_now,237583333,1
3,11/07/1984,235973000,300000000,tashkent_city,M,married,no_credit_now,248283333,1
4,25/03/1991,452710700,700000000,andijan,M,married,no_credit_now,3819750000,1
...,...,...,...,...,...,...,...,...,...
3306,25/01/1980,459650000,200000000,sirdarya,F,married,no_credit_now,203833333,1
3307,05/06/1988,429042000,150000000,tashkent_city,M,married,no_credit_now,161900000,1
3308,22/05/1973,318780000,280000000,tashkent_city,F,not_married,has_credit_now,504333333,1
3309,15/10/1996,329709600,440000000,tashkent,M,married,has_credit_now,165600000,1


In [8]:
data.dtypes

birth_date             object
credit_amount           int64
salary_amount           int64
region                 object
gender                 object
marital_status         object
credit_history         object
card_incomes_amount    object
target                  int64
dtype: object